<a href="https://colab.research.google.com/github/pgurazada/prompt-engineering/blob/main/prompt-engineering-with-azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Case Study -  Khanmigo!

Khanmigo is an AI-powered tool from Khan Academy that they call "Tutor for learners. Assistant for teachers.” It provides personalized learning experiences for students. It offers guidance to students on math, science, and humanities problems, a debate tool, and a writing tutor. It can support teachers in several tasks, like a teaching assistant.

Today, we are going to replicate some of the features of Khanmigo using prompting techniques we have learnt so far. We are going to focus on select features of Khanmigo that are developed to help teachers.

We will step into the shoes of developers who have built these features for Khanmigo.

## Journey of a Teacher

To Develop features that a teacher would require, we need to empathise with the teacher. So, Let's jump into the shoes of a teacher. Let's say Our first assignment is to teach about Newton's third law of motion to grade 8 students. Below is the lesson material provided by our institution.

**Lesson Material:**

Title: Understanding Newton's Third Law: Action and Reaction

**Objective:**
By the end of this lesson, students will be able to:

1. Define Newton's Third Law of Motion.
2. Identify action and reaction force pairs in various scenarios.
3. Explain real-life applications of Newton's Third Law.

**Body:**

**1. What is Newton's Third Law?**
   - Explain that Newton's Third Law states: "For every action, there is an equal and opposite reaction."
   - Break down the meaning of this law: when one object exerts a force on a second object, the second object exerts a force of equal magnitude in the opposite direction on the first object.

**2. Action and Reaction Pairs:**
   - Provide examples and ask students to identify the action and reaction force pairs in each scenario. Examples could include:
     - A person pushing against a wall.
     - A ball bouncing off the ground.
     - A swimmer pushing against the water.
   - Encourage students to think about the forces involved and how they act on different objects.

**3. Real-Life Applications:**
   - Discuss real-life examples where Newton's Third Law is at play, such as:
     - Rocket propulsion: The action force is the expulsion of gas from the rocket engines, and the reaction force is the resulting upward movement of the rocket.
     - Walking: Each step involves pushing against the ground, which pushes back with an equal force, propelling the person forward.
     - Driving a car: The tires push against the road, and the road pushes back, propelling the car forward.
   - Encourage students to brainstorm other examples they encounter in daily life.
   
**Conclusion:**
Review the key points of the lesson:
- Newton's Third Law states that for every action, there is an equal and opposite reaction.
- Action and reaction force pairs exist in all interactions between objects.
- Understanding Newton's Third Law helps us explain and predict motion in the world around us.

**Assessment:**
Conduct a short quiz or discussion to assess students' understanding of Newton's Third Law and their ability to identify action and reaction force pairs in different scenarios. Encourage students to ask questions if they need further clarification.

**Extension:**
Challenge students to research and present on advanced applications of Newton's Third Law, such as in sports, engineering, or space exploration. This can deepen their understanding and appreciation of the concept.



---


This is good foundational material but this alone doesn't provide great learning experience. One of the first things a good learning experience requires is a lesson hook.

A lesson hook is a tool used by teacher to grab students' attention and maintaining their engagement, providing a gateway into the lesson's content.

A lesson hook can take various forms, such as a question, activity, game, story, or compelling analogy. Each topic can be approached and explained from diverse angles. An effective lesson hook not only seizes attention but also makes the subject matter relevant to students' lives, sparking curiosity and fostering a desire to delve deeper into the topic.

However, brainstorming lesson hooks for each topic can be overwhelming, particularly when dealing with multiple classes or classes with varying cognitive levels. Leveraging Language Models (LLMs), teachers can swiftly generate ideas tailored to the topic and cognitive level. Additionally, incorporating students' interests and hobbies can further enhance the relevance and relatability of the lesson hooks.


---



So, the first feature of our app should be a 'lesson hook creator'. Before we dive into building that, let's set up our environment.

## Setup

### Installation

Since Colab instances are not persistent, for each session, we need to install the packages required for the session each time a new session is started.


In [1]:
!pip install openai==1.2 tiktoken==0.6 session-info --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.14 requires openai<2.0.0,>=1.32.0, but you have openai 1.2.0 which is incompatible.
langchain-openai 0.1.14 requires tiktoken<1,>=0.7, but you have tiktoken 0.6.0 which is incompatible.


### Imports

In [2]:
# Import all Python packages required to access the Azure Open AI API
import json
import tiktoken
import session_info

from openai import AzureOpenAI

In [3]:
session_info.show()

### Authentication

In order to be abe to authenticate our identity and programmatically access the Azure Open AI API from Google Colab, we need to read our credentials into the Colab environment in a secure way.

A strongly recommended method to do this is to write credentials to a `json` file that is read during execution of a Colab session. Do not save keys in a public repository.

In [48]:
# Azure Open AI redentials and the id of the deployed chat model are stored as
# key value pairs in a json file

with open('config_4o.json', 'r') as az_creds:
    data = az_creds.read()

In [42]:
with open('4omini.json', 'r') as az_creds:
    data = az_creds.read()

In [49]:
creds = json.loads(data)

In [50]:
# Credentials to authenticate to the personalized Open AI model server

client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [51]:
# Deployment namre of the ChatCompletion endpoint
deployment_name = creds["CHATGPT_MODEL"]

## 1 Creating a Lesson Hook

**What Does a Lesson Hook Accomplish?**

**Grabbing Attention:** A hook immediately captures students' attention and focuses their minds on the lesson at hand.

**Generating Curiosity:** It piques students' curiosity and encourages them to want to learn more about the topic.

**Creating Relevance:** By relating the lesson to students' lives or interests, a hook makes the material more relevant and meaningful to them.

**Building Enthusiasm:** A well-executed hook creates excitement and enthusiasm for learning, motivating students to actively participate in the lesson.

**Establishing Connection:** It establishes a connection between the students and the lesson content, laying the foundation for deeper understanding and engagement.

In summary, a lesson hook is a powerful tool for engaging students, generating interest, and setting the stage for effective learning. It captures attention, fosters curiosity, and makes the lesson content relevant and meaningful to students, ultimately leading to greater retention and understanding.



---


We need to create a prompt that can get a lesson hook from the LLM given a topic. Of the three prompting techniques we learnt, what do you think is the best prompting technique for this purpose?

Let's try one by one and see what works best. You are encouraged to suggest modifications to the prompts.

---



### 1.1 Zero-Shot

In [31]:
zero_shot_prompt = """Create a list of 3-5 engaging lesson hooks for a  lesson keeping in mind the grade of the students.
The hooks should be designed to capture students' attention and interest at the start of the lesson.
Consider the developmental needs and interests of students at this age when crafting the hooks."""

In [32]:
user_prompt = "Create a lesson hook for a lesson on newton's third law of motion for grade 8 students"

In [33]:
messages = [
    {"role": "system", "content": zero_shot_prompt},
    {"role": "user", "content": user_prompt}
]

In [52]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=4096
)

In [53]:
print(response.choices[0].message.content)

1. **Balloon Rocket Experiment**: Begin the lesson by setting up a balloon rocket demonstration. Inflate a balloon and tape it to a straw threaded on a string stretched across the classroom. Release the balloon and let it zoom across the string. Ask the students to observe and guess what just happened. Use this activity to introduce the concept of action and reaction forces present in Newton's Third Law of Motion.

2. **Recoil Challenge**: Bring in a simple toy like a Nerf gun or a rubber band launcher. Let a student or volunteer hold the gun still, and shoot a foam dart or stretch and release the rubber band. Immediately ask the class to describe any movement they saw. Why did the toy move backward a little? Discuss the idea of how actions cause reactions, linking it to everyday experiences like firing a gun or kicking a soccer ball.

3. **Video Clip Hook**: Show a short and engaging video clip of a rocket launch or a skateboard trick. Pause the clip at an exciting moment and ask the 

That's not a bad output at all. We can further modify parameters like temperature and top-k.

What do you think temperature should be for this excercise. Low or High? Why do you think so?

### 1.2 Temperature

This parameter controls the variability in the response. Using a higher temperature results in potentially lower probability tokens being chosen.

With higher temperatures, we could prompt the model to make multiple completions for the same prompt.

With a temperature of 0, only the highest probability tokens are chosen so multiple completions should return the same response. This is not very useful for our feature. We want to give multiple options to the teachers to choose from.

In [15]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    n=2,# <- This will ask the model to present two choices for completion so that the teacher has many options to choose from
    temperature=0
)

In [16]:
print(response.choices[0].message.content)

1. "The Great Balloon Race": Inflate two balloons and attach them to a string. Have students hold the string and let go of the balloons at the same time. Ask them to observe what happens. This activity demonstrates Newton's Third Law of Motion, where for every action, there is an equal and opposite reaction.

2. "Egg Drop Challenge": Challenge students to design a container that will protect an egg from breaking when dropped from a height. This activity will require students to apply Newton's Third Law of Motion to create a design that will counteract the force of gravity and prevent the egg from breaking.

3. "Human Tug-of-War": Divide the class into two teams and have them play a game of tug-of-war. Ask students to observe what happens when one team pulls harder than the other. This activity demonstrates Newton's Third Law of Motion, where the force exerted by one team is equal and opposite to the force exerted by the other team.

4. "Rocket Launch": Build a simple rocket using a bal

In [17]:
print(response.choices[1].message.content)

1. "The Great Balloon Race": Inflate two balloons and attach them to a string. Have students hold the string and let go of the balloons at the same time. Ask them to observe what happens. This activity demonstrates Newton's Third Law of Motion, where for every action, there is an equal and opposite reaction.

2. "Egg Drop Challenge": Challenge students to design a container that will protect an egg from breaking when dropped from a height. This activity will require students to apply Newton's Third Law of Motion to create a design that will counteract the force of gravity and prevent the egg from breaking.

3. "Human Tug-of-War": Divide the class into two teams and have them play a game of tug-of-war. Ask students to observe what happens when one team pulls harder than the other. This activity demonstrates Newton's Third Law of Motion, where the force exerted by one team is equal and opposite to the force exerted by the other team.

4. "Rocket Launch": Build a simple rocket using a bal

In [18]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=4096,
    n=2, # <- This will ask the model to present two choices for completion
    temperature=0.8
)

In [19]:
print(response.choices[0].message.content)

1. "The Ultimate Reaction Time Challenge": Set up a table tennis game in the classroom, but instead of using regular paddles, use paper plates and popsicle sticks to create makeshift paddles. Divide the students into teams and challenge them to hit the ball with their "paddles" as quickly as they can. After a few rounds, introduce the concept of Newton's Third Law by showing how the ball reacts with equal and opposite force to the paddle's impact.

2. "Human Newton's Cradle": Using tennis balls or any other small balls, set up a human-sized version of Newton's Cradle in the classroom. Assign students to stand in a line and hold hands. After dropping one ball, watch as the energy from the ball is transferred down the line, causing the last person to release the final ball. Use this demonstration to introduce the concept of conservation of energy and how Newton's Third Law applies.

3. "Egg Drop Challenge": Divide the class into small groups and provide them with a raw egg and various ma

In [20]:
print(response.choices[1].message.content)

1. Egg Drop Challenge: Start the lesson by challenging students to design a container that will protect an egg from breaking when dropped from a height. Once students have designed their containers, have them drop the containers and observe the impact. Then ask students to explain why the egg did or did not break, and how Newton's third law of motion applies to their designs.

2. Balloon Rockets: Set up a simple experiment by attaching a balloon to a string and then releasing the air. The balloon will act as a rocket and move in the opposite direction of the air being released. Ask students to explain why the balloon moves in this way, and how Newton's third law of motion applies to the experiment.

3. Human Tug of War: Divide the class into two teams and have them play a game of tug of war. Ask students to explain why both teams are pulling with equal force, and how Newton's third law of motion applies to the game.

4. Ping Pong Ball Launcher: Build a simple ping pong ball launcher us

Those are decent results. Moving on, let's look at the quality of results we can get with few-shot prompting.

### 1.3 Few Shot Prompting

In [21]:
system_message = """Create a list of 3-5 engaging lesson hooks for a  lesson keeping in mind the grade of the students.
The hooks should be designed to capture students' attention and interest at the start of the lesson.
Consider the developmental needs and interests of students at this age when crafting the hooks."""

In [23]:
messsages = [
    {"role": "system", "content": system_message},

    {"role": "user", "content": "Create a lesson hook on States of Matter for grade 5"},
    {"role": "assistant", "content": """Bring in a few different objects (e.g., ice cube, piece of metal, balloon) and ask students to describe their properties.
Then reveal that these objects are all made of the same basic building blocks - atoms and molecules - just in different states."""},

    {"role": "user", "content": "Create a lesson hook on Multiplication for grade 3"},
    {"role": "assistant", "content": """Tell a simple story about going to the store and buying multiple items of the same kind (e.g., 3 bags of apples with 4 apples in each bag).
Have students act it out with manipulatives to discover multiplication."""},

    {"role": "user", "content": "Create a lesson hook on civil war for grade 7"},
    {"role": "assistant", "content": """Hook: Read contrasting first-hand accounts from soldiers on opposite sides of a major battle.
Discuss how perspectives differ."""},

    {"role": "user", "content": "Create a lesson hook on Photosynthesis for grade 7"},
    {"role": "assistant", "content": """The Incredible Shrinking Student: Tell students to imagine they are shrinking down to the size of a molecule to take a journey inside a leaf.
Use this to introduce the structures involved in photosynthesis."""},

    {"role": "user", "content": "Create a lesson hook on Photosynthesis for grade 7"},
    {"role": "assistant", "content": """Divide the class into teams, each representing a part of the photosynthesis process.
Have them race to complete the process in the correct order."""},

    {"role": "user", "content": "Create a lesson hook on Anantomy for grade 6"},
    {"role": "assistant", "content": """"Interview a Body Part": Assign each student a specific body part or system.
Have them create interview questions and then act out an interview with their assigned part, discussing its functions, challenges, and fun facts.
"""},

    {"role": "user", "content": "Create a lesson hook on Ancient Civilizations for grade 7"},
    {"role": "assistant", "content": """""The Lost Civilization": Students are archaeologists who have discovered a long-lost civilization.
As they uncover artifacts and decipher ancient texts, they learn about the culture, achievements, and eventual decline of this mysterious society, inspiring a deep dive into ancient history.
Question Hook: "Why do some societies thrive while others collapse? Is it simply a matter of luck, or are there underlying patterns?" This hook is ideal for a unit on the rise and fall of ancient civilizations.
"""},
    {"role": "user", "content": user_prompt}
    ]


In [24]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    n=2,# <- This will ask the model to present two choices for completion
    temperature=0.8
)

In [25]:
print(response.choices[0].message.content)

1. "Egg Drop Challenge": Begin by showing the students a raw egg and ask them if they think it will break if you drop it on the ground. Then, ask them what will happen if you drop an egg on a hard floor from a greater height. This will get students thinking about force and motion. Next, divide the class into groups and give them materials such as straws, paper, tape, and a plastic cup. Challenge the students to create a device that will protect the egg when dropped from a height. This activity will demonstrate Newton's third law of motion that for every action, there is an equal and opposite reaction.

2. "Balloon Rockets": Inflate balloons and attach them to a string that is stretched across the classroom. Have students tape a straw to the balloon, attach a long piece of string to the straw, and hold the other end of the string. When the balloon is released, the air rushes out of the balloon in one direction, causing the balloon to move in the opposite direction. This activity will he

In [26]:
print(response.choices[1].message.content)

1. "Newton's Third Law Dance-Off": Start the lesson by playing upbeat music and inviting students to come up with their best dance moves. Have them dance in pairs or groups and explain that they will be learning about Newton's Third Law of Motion, which talks about the equal and opposite reactions that happen when two objects interact. Ask them to think about how their dance moves demonstrate this law.

2. "Marshmallow Catapult Challenge": Bring in materials for students to make mini catapults using popsicle sticks, rubber bands, and spoons. Have them launch mini marshmallows and observe the equal and opposite forces at play. Ask them to think about how changing the force of the catapult or the weight of the marshmallow affects the reaction.

3. "Newton's Third Law of Motion in Sports": Show clips of athletes in action, such as a basketball player shooting a ball or a football player tackling someone. Explain how Newton's Third Law of Motion applies to these actions, with the player ex

The performance of few-shot prompting is tied to the examples provided in the prompt and also on the nature of the task. In this task, few-shot is not significantly better than zero-shot. Better examples could lead to better results.

Using examples, you can drive guide the LLM's response in new directions. You can use as many examples as you wish, however, with each new example you are increasing the token consumption per API call which will lead to higher costs. The number of examples should be optimised by iteratively adding and removing examples and checking its effect on performance.

Checking performance on a generation task can be challenging. In the next week, we will do the same exercise of optimising examples in a few-shot prompt for a classification task where it is easier to guage the performance.

At the start, we said a lesson hook should have the following qualities.

1. Grabbing Attention

2. Generating Curiosity

3. Creating Relevance

4. Building Enthusiasm

5. Establishing Connection

We can use enforce these qualities in the llm's response by crafting a chain of thought prompt.

### 1.4 CoT

In [50]:
cot_prompt = """
 Create an engaging lesson hooks for a topic for a given grade level classroom.
 The hooks should be designed to capture students' attention and interest at the start of the lesson.
 Consider the developmental needs and interests of students at this age when crafting the hooks.

Below are 5 steps you need to follow to craft the lesson hook. At each step, you should check if any rewriting of the hook is necessary. 
If so, add or remove text to aligh with the step. At the end, you should give me a lesson hook that follows all the guidelines.
At each step, evaluate if the current core of the hook is effective. 
If at any point you feel like the current hook is ineffective, you should restart crafting the hook with a new core idea.

Step 1: Identify the lesson topic and key concepts
- What is the main subject or theme of the lesson?
- What are the most important ideas or skills students should learn?

Step 2: Consider the students' interests, experiences, and prior knowledge
- What do the students already know about this topic?
- What aspects of the topic might the students find most interesting or relatable to their lives?
- What current events, popular culture references, or real-world examples could help connect the lesson to the students' experiences?

Step 3: Brainstorm attention-grabbing opening techniques
- Could you start with a surprising fact, statistic, or statement related to the topic?
- Is there a thought-provoking question you could pose to the students?
- Could you present a mysterious or intriguing scenario that ties into the lesson?
- Would a short, engaging story or anecdote help capture the students' attention?

Step 4: Incorporate elements of curiosity and relevance
- How can you frame the hook to make students curious to learn more about the topic?
- What aspects of the hook will make the lesson feel relevant and meaningful to the students' lives or interests?
- Could you hint at an exciting activity, demonstration, or revelation to come later in the lesson?


At the end, evaluate all the ideas you have created and finally choose a single lesson hook that's best suited. 
Explain why it's the best lesson hook and then write the best lesson hook.
"""

In [51]:
user_prompt = "Create a lesson hook for a lesson on newton's third law of motion for grade 8 students"

In [52]:
messages = [
    {"role": "system", "content": cot_prompt},
    {"role": "user", "content": user_prompt}
]

In [53]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    n=2,# <- This will ask the model to present two choices for completion
    temperature=0.6
)

In [54]:
print(response.choices[0].message.content)

Step 1: 
- Lesson topic: Newton's Third Law of Motion
- Key concepts: Action and reaction forces, equal and opposite forces, force pairs

Step 2: 
- Students may have prior knowledge of Newton's Laws of Motion but may not be familiar with the third law specifically.
- Students may find it interesting to learn that every action has an equal and opposite reaction.
- Real-world examples such as bouncing balls, rockets launching, and car crashes could help connect the lesson to the students' experiences.

Step 3: 
- Start with a demonstration of a balloon rocket. Inflate a balloon and tape it to a straw. Thread a long string through the straw and hold each end of the string. When the balloon is released, it will move forward, demonstrating Newton's Third Law of Motion.
- Pose a thought-provoking question such as "Have you ever wondered why you feel a push back when you push something forward?"
- Share an engaging story or anecdote about a famous scientist who discovered the law.

Step 4: 


In [55]:
print(response.choices[1].message.content)

Step 1: 
- The main subject of the lesson is Newton's Third Law of Motion.
- The key concepts are action and reaction forces, and how they are equal in magnitude and opposite in direction.

Step 2:
- The students may already have some knowledge of Newton's laws of motion from previous science classes.
- Students may find it interesting to learn how everyday actions involve Newton's Third Law, such as jumping on a trampoline or hitting a ball with a bat.
- Popular culture references, such as superhero movies, may help connect the lesson to the students' experiences.

Step 3:
- Start with a demonstration: drop two objects of different weights and ask students which one will hit the ground first. Then, drop them at the same time and ask if the result was expected. Explain that the reason both objects hit the ground at the same time is due to Newton's Third Law.
- Ask students to stand up and try to push against a wall. Then ask them to push against a partner and see what happens. Explain 

We got lesson hooks that are more comprehensive but not necessarily better solution for teachers. The developer should work with teachers to understand whether they prefer a collection creative hooks to choose from for or a single lesson hook that is more comprehensive. Teacher input should guide you towards the right prompting technique. You can once again experiment with parameters to find the best combination. As we have said earlier, prompting is an interative excersice.

## 2 Crafting Examples

The lesson hook engages the students and makes the class exciting and interactive. However, we also need to provide ample amount of examples to make the learning relevant to the real world.

Teachers often do not have the time to brainstorm real world examples. LLMs are of great help here. A list of examples can also be provided as additional material for the students. 



---


What do you think is a sufficient prompting technique to craft examples for a topic?


---


While it's often observed that Chain of though (COT) and Few-Shot prompting tend to yield superior results compared to Zero-Shot Learning, it's important to note their token-intensive nature, which invariably leads to higher costs. For companies like Khanmigo, operating on a subscription-based revenue model, mitigating such expenses is crucial. In instances where Zero-Shot Learning demonstrates comparable performance, it should be the preferred option.

In the context of crafting examples, Zero-Shot prompting stands out as an apt choice.

However, give that this has a creative element to it, what do you think should be the temperature, top-k, top-p should be for this exercise? High or Low ?

### 2.1 Zero Shot

In [60]:
poem_prompt = """Write 10 examples for the following lesson material. Add a brief explanation of the example.
The lesson material is:

{material}

"""

In [61]:
user_input = """
**Lesson Material:**

Title: Understanding Newton's Third Law: Action and Reaction

**Objective:**
By the end of this lesson, students will be able to:

1. Define Newton's Third Law of Motion.
2. Identify action and reaction force pairs in various scenarios.
3. Explain real-life applications of Newton's Third Law.

**Body:**

**1. What is Newton's Third Law?**
   - Explain that Newton's Third Law states: "For every action, there is an equal and opposite reaction."
   - Break down the meaning of this law: when one object exerts a force on a second object, the second object exerts a force of equal magnitude in the opposite direction on the first object.

**2. Action and Reaction Pairs:**
   - Provide examples and ask students to identify the action and reaction force pairs in each scenario. Examples could include:
     - A person pushing against a wall.
     - A ball bouncing off the ground.
     - A swimmer pushing against the water.
   - Encourage students to think about the forces involved and how they act on different objects.

**3. Real-Life Applications:**
   - Discuss real-life examples where Newton's Third Law is at play, such as:
     - Rocket propulsion: The action force is the expulsion of gas from the rocket engines, and the reaction force is the resulting upward movement of the rocket.
     - Walking: Each step involves pushing against the ground, which pushes back with an equal force, propelling the person forward.
     - Driving a car: The tires push against the road, and the road pushes back, propelling the car forward.
   - Encourage students to brainstorm other examples they encounter in daily life.
   
"""

In [62]:
response = client.chat.completions.create(
    messages = [{"role": "user", "content": poem_prompt.format(material = user_input)}],
    model = deployment_name,
    max_tokens=1000,
    temperature=0.8
)

In [63]:
print(response.choices[0].message.content)

1. Example: When a soccer player kicks a ball, the ball moves forward with a certain force. In turn, the player's foot experiences an equal and opposite force, pushing back against the foot.
Explanation: This example illustrates Newton's Third Law by showing how the action of kicking the ball creates a reaction force in the opposite direction.

2. Example: A person sitting on a chair exerts a force downward on the chair, while the chair exerts an equal and opposite force upward on the person.
Explanation: This example demonstrates how action and reaction force pairs work in everyday life and helps students identify these pairs.

3. Example: When a person jumps off a diving board, they push down against the board, which then pushes back up with an equal and opposite force.
Explanation: This example shows how the force exerted by the person jumping creates a reaction force that propels them upward.

4. Example: When a car collides with a wall, the car exerts a force on the wall, while th

These are nice examples but they are very common ones. The teacher can come up with such examples themselves. Let's try to get examples that are uncommon. One way to do that is by making the LLM chose uncommon words at each step. Let's set the top-p high so that the model choses new low probabilty words as well.

### 2.2 Top p

In [64]:
response = client.chat.completions.create(
    messages = [{"role": "user", "content": poem_prompt.format(material = user_input)}],
    model = deployment_name,
    max_tokens=1000,
    temperature=0.95,
    top_p = 0.6
)

In [65]:
print(response.choices[0].message.content)

1. When a person jumps off a diving board, the force of their jump is the action force, and the reaction force is the force of the diving board pushing back up on the person.
2. When a hammer strikes a nail, the action force is the force of the hammer hitting the nail, and the reaction force is the force of the nail pushing back up on the hammer.
3. When a person pulls on a rope, the action force is the force of the person pulling on the rope, and the reaction force is the force of the rope pulling back on the person.
4. When a bird flaps its wings, the action force is the force of the bird pushing down on the air, and the reaction force is the force of the air pushing up on the bird.
5. When a car crashes into a wall, the action force is the force of the car hitting the wall, and the reaction force is the force of the wall pushing back on the car.
6. When a person walks on the ground, the action force is the force of the person's foot pushing against the ground, and the reaction force

Good! However, we might want to use newer contexts to make it novel. So increasing one of frequency penalty and presence penaltys should do the trick. (Changing the prompt will also works here. But let's see how frequency penalty individually affects the response.)

presence_penalty: Optional - Number between -2.0 and 2.0. - Defaults to 0
Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

frequency_penalty - Optional - Number between -2.0 and 2.0. - Defaults to 0
Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

Let's set the frequency penalty high so that we get new words.

### 2.3 Penalties

In [66]:
response = client.chat.completions.create(
    messages = [{"role": "user", "content": poem_prompt.format(material = user_input)}],
    model = deployment_name,
    max_tokens=1000,
    temperature=0.95,
    frequency_penalty = 1.5
)

In [67]:
print(response.choices[0].message.content)

1. When a person throws a ball, the ball exerts an equal and opposite force back on the person's hand.
2. When a bird flaps its wings, it creates an action force that propels air downwards, creating a reaction force that lifts the bird upwards.
3. When a swimmer dives into water, they exert an action force on the water which creates a reaction force that propels them forward.
4. A stuffed animal lying on top of another stuffed animal will exert weight and cause the bottom stuffed animal to press upward with equal pressure known as buoyancy or flotation.
5. Jumping off from trampolines: The trampoline mat gets depressed because of your weight which in turn pushes you in higher direction with equal energy causing you to jump up again.
6. Playing soccer: kicking motion is actually pushing against the ground backward while making contact with football balls pushing forward towards goal post causing balance in motions ).
7.The recoil energy from firing guns leads to Newton’s Third Law where

Slight Improvement. Let's also change the prompt. 

In [68]:
poem_prompt = """Write 10 uncommon examples for the following lesson material. 
Add a brief explanation of the example.
The lesson material is:

{material}

"""

In [69]:
response = client.chat.completions.create(
    messages = [{"role": "user", "content": poem_prompt.format(material = user_input)}],
    model = deployment_name,
    max_tokens=1000,
    temperature=0.95,
    frequency_penalty = 1.5
)

In [70]:
print(response.choices[0].message.content)

1. A car crashing into a wall: The action force is the car's forward momentum, and the reaction force is the impact of the collision that pushes back on the car.
2. Jumping on a trampoline: The action force comes from jumping down onto the trampoline, and the reaction force propels you back upwards.
3. Swinging a bat: The action force comes from swinging the bat towards an object, and when contact is made, there is an equal but opposite reaction as felt by your hands.
4. Launching fireworks: The explosive chemicals inside create an action force pushing outwards in all directions which results in various visual effects (reaction forces), such as colorful explosions or shooting stars across sky.
5. Kicking a football/punching bag/etc.: Every time one kicks or hits something rotating their hips/body sideways creating torsion spring like effect helping produce maximum speed & power transfer to ball/bag/other object while also experiencing reactionary forces - preventing them from falling o

## 3 Automating Quizzes




---


Once the lesson is taught, the subsequent task for a teacher involves crafting a test to assess students' retention. Generating quizzes can be a tedious endeavor. Let's leverage Language Models to automate the quiz creation process.


---



### 3.1 Zero_shot

In [ ]:
quiz_input = """Task: Generate a mcq quiz on the teaching material mentioned above


Output Format:
Use the following format for each question:

Question: [QUESTION TEXT]
Options:
a) [OPTION A]
b) [OPTION B]
c) [OPTION C]
d) [OPTION D]

Correct option: [Correct Option]

Incorrect Answers:
a) [EXPLANATION FOR OPTION A BEING INCORRECT]
b) [EXPLANATION FOR OPTION B BEING INCORRECT]
c) [EXPLANATION FOR OPTION C BEING INCORRECT]
d) [EXPLANATION FOR OPTION D BEING INCORRECT]

"""

In [ ]:
user_input = """
**Lesson Material:**


Title: Understanding Newton's Third Law: Action and Reaction

**Objective:**
By the end of this lesson, students will be able to:

1. Define Newton's Third Law of Motion.
2. Identify action and reaction force pairs in various scenarios.
3. Explain real-life applications of Newton's Third Law.

**Body:**

**1. What is Newton's Third Law?**
   - Explain that Newton's Third Law states: "For every action, there is an equal and opposite reaction."
   - Break down the meaning of this law: when one object exerts a force on a second object, the second object exerts a force of equal magnitude in the opposite direction on the first object.

**2. Action and Reaction Pairs:**
   - Provide examples and ask students to identify the action and reaction force pairs in each scenario. Examples could include:
     - A person pushing against a wall.
     - A ball bouncing off the ground.
     - A swimmer pushing against the water.
   - Encourage students to think about the forces involved and how they act on different objects.

**3. Real-Life Applications:**
   - Discuss real-life examples where Newton's Third Law is at play, such as:
     - Rocket propulsion: The action force is the expulsion of gas from the rocket engines, and the reaction force is the resulting upward movement of the rocket.
     - Walking: Each step involves pushing against the ground, which pushes back with an equal force, propelling the person forward.
     - Driving a car: The tires push against the road, and the road pushes back, propelling the car forward.
   - Encourage students to brainstorm other examples they encounter in daily life.
   
**Conclusion:**
Review the key points of the lesson:
- Newton's Third Law states that for every action, there is an equal and opposite reaction.
- Action and reaction force pairs exist in all interactions between objects.
- Understanding Newton's Third Law helps us explain and predict motion in the world around us.
"""

In [ ]:
messages = [{"role":"user","content":user_input+quiz_input}]

In [ ]:
response = client.chat.completions.create(
    messages = messages,
    model = deployment_name,
    max_tokens=1000,
    temperature=0.2
)

In [ ]:
print(response.choices[0].message.content)

Question: Where was Christopher Columbus born?
Options:
a) Madrid, Spain
b) Genoa, Italy
c) Paris, France
d) London, England

Correct option: b) Genoa, Italy

Incorrect Answers:
a) Madrid is the capital of Spain, not the birthplace of Columbus.
c) Paris is the capital of France, not the birthplace of Columbus.
d) London is the capital of England, not the birthplace of Columbus.

Question: What was Columbus's goal in sailing west across the Atlantic Ocean?
Options:
a) To find a faster route to Africa
b) To find a faster route to Asia
c) To find a faster route to South America
d) To find a faster route to Australia

Correct option: b) To find a faster route to Asia

Incorrect Answers:
a) Columbus was not seeking a faster route to Africa.
c) Columbus did not know about South America at the time of his voyages.
d) Columbus was not seeking a route to Australia.

Question: Who funded Columbus's voyage in 1492?
Options:
a) King Henry VIII of England
b) Queen Elizabeth I of England
c) Queen Is

### 3.2 COT

Let's try COT with this exercise. We would expect COT to perform better.

In [ ]:
quiz_input = """Task: Generate a quiz on the teaching material mentioned above with the following specifications:

First:
- Understand the key concepts related to teaching material
- Identify the components/elements of teaching material
- Apply the principles of teaching material to solve problems

Question Format:
- Generate 10 multiple-choice questions with 4 options each
- Ensure questions are clear, concise, and test only one concept/skill
- Avoid ambiguity or tricky language in the questions and options

Options:
- Include relevant and plausible distractors (incorrect options)
- Vary the length of options to prevent unintentional hints
- Randomize the order of options for each question

Explanations:
- Provide a detailed explanation for the correct answer, including additional context, examples, or reinforcement of understanding
- For each incorrect option, explain why it is incorrect and address any potential misconceptions or errors in thinking

Output Format:
Use the following format for each question:

Question: [QUESTION TEXT]
Options:
a) [OPTION A]
b) [OPTION B]
c) [OPTION C]
d) [OPTION D]

Explanation:
[EXPLANATION FOR CORRECT ANSWER]

Incorrect Answers:
a) [EXPLANATION FOR OPTION A BEING CORRECT/INCORRECT]
b) [EXPLANATION FOR OPTION B BEING CORRECT/INCORRECT]
c) [EXPLANATION FOR OPTION C BEING CORRECT/INCORRECT]
d) [EXPLANATION FOR OPTION D BEING CORRECT/INCORRECT]

Think step by step"""

In [ ]:
user_input = """
**Lesson Material:**


Title: Understanding Newton's Third Law: Action and Reaction

**Objective:**
By the end of this lesson, students will be able to:

1. Define Newton's Third Law of Motion.
2. Identify action and reaction force pairs in various scenarios.
3. Explain real-life applications of Newton's Third Law.

**Body:**

**1. What is Newton's Third Law?**
   - Explain that Newton's Third Law states: "For every action, there is an equal and opposite reaction."
   - Break down the meaning of this law: when one object exerts a force on a second object, the second object exerts a force of equal magnitude in the opposite direction on the first object.

**2. Action and Reaction Pairs:**
   - Provide examples and ask students to identify the action and reaction force pairs in each scenario. Examples could include:
     - A person pushing against a wall.
     - A ball bouncing off the ground.
     - A swimmer pushing against the water.
   - Encourage students to think about the forces involved and how they act on different objects.

**3. Real-Life Applications:**
   - Discuss real-life examples where Newton's Third Law is at play, such as:
     - Rocket propulsion: The action force is the expulsion of gas from the rocket engines, and the reaction force is the resulting upward movement of the rocket.
     - Walking: Each step involves pushing against the ground, which pushes back with an equal force, propelling the person forward.
     - Driving a car: The tires push against the road, and the road pushes back, propelling the car forward.
   - Encourage students to brainstorm other examples they encounter in daily life.
   
**Conclusion:**
Review the key points of the lesson:
- Newton's Third Law states that for every action, there is an equal and opposite reaction.
- Action and reaction force pairs exist in all interactions between objects.
- Understanding Newton's Third Law helps us explain and predict motion in the world around us.

"""

In [ ]:
messages = [{"role":"user","content":user_input+quiz_input}]

In [ ]:
response = client.chat.completions.create(
    messages = messages,
    model = deployment_name,
    max_tokens=1000,
    temperature=0.2
)

In [ ]:
print(response.choices[0].message.content)

Question 1: Where was Christopher Columbus born?
Options:
a) Madrid, Spain
b) Paris, France
c) Genoa, Italy
d) London, England

Explanation:
The correct answer is c) Genoa, Italy. Christopher Columbus was born in Genoa, Italy, around 1451.

Incorrect Answers:
a) Madrid, Spain is incorrect because Columbus was not born in Spain.
b) Paris, France is incorrect because Columbus was not born in France.
d) London, England is incorrect because Columbus was not born in England.

Question 2: What was Christopher Columbus's dream?
Options:
a) To find a faster route to Asia by sailing west across the Atlantic Ocean
b) To become a famous painter
c) To discover a new continent
d) To become a wealthy merchant

Explanation:
The correct answer is a) To find a faster route to Asia by sailing west across the Atlantic Ocean. Columbus became fascinated with the idea of exploration and dreamed of finding a faster route to Asia by sailing west across the Atlantic Ocean.

Incorrect Answers:
b) To become a fa

The output from both the techniques is not that different. COT prompting isn't as useful as we expected in this situation. Zero-shot-prompt performed suffieciently well and since it is token-efficient, we shoul use that. However, in a different context COT's performance can be substantially higher.

For example, let's carry out the same exercise for a math class. Let's say the topic is solving two-step equations.

## 4 Math Quiz

**Things to Ponder:**
What kind of temperature, top-p, frequency penalty values can we use here?

Note: Though we want the answers to be same everytime, we want different questions so that the teacher can chose from a variety of options.


What happens if we change temperature and what happens if we change top-p?

Temperature regulates randomness, with lower temperatures yielding less randomness (greater determinism), whereas higher temperatures result in increased randomness.

In more technical terms, a lower temperature boosts the model's confidence in its primary selections. A temperature set at 0 equates to argmax or maximum likelihood, representing the token with the highest probability. A higher temperature modifies the probabilities in such a way that all probabilities are closer to each other i.e., the model has only slightly more confidence in it's top choices. We do not want the model to be in this state for calculation. Hence a temperature of zero is fit for this situation.

Regarding top-p, it calculates the cumulative probability distribution and halts once this distribution surpasses the specified top-p value. For instance, a top-p value of 0.3 implies that only tokens encompassing the top 30% probability mass are considered.


Between the two options, it is far more logical to maintain a temperature of zero. This ensures that high-probability terms, such as answers, are prioritized for selection. In instances where high probability terms are not present, words can be chosen from a limited sample set by incrementally increasing the top-p to small values.

### 4.1 Zero_shot

In [ ]:
quiz_input = """Task: Generate a mcq quiz on the teaching material mentioned above.


Output Format:
Use the following format for each question:

Question: [QUESTION TEXT]
Options:
a) [OPTION A]
b) [OPTION B]
c) [OPTION C]
d) [OPTION D]

Correct Option: [Correct Option]

Explanation:
[EXPLANATION FOR CORRECT ANSWER]

Incorrect Answers:
a) [EXPLANATION FOR OPTION A BEING CORRECT/INCORRECT]
b) [EXPLANATION FOR OPTION B BEING CORRECT/INCORRECT]
c) [EXPLANATION FOR OPTION C BEING CORRECT/INCORRECT]
d) [EXPLANATION FOR OPTION D BEING CORRECT/INCORRECT]

"""

In [ ]:
user_input = """
**Lesson Material:**

solving two-step equations"""

In [ ]:
messages = [{"role":"user","content":user_input+quiz_input}]

In [ ]:
response = client.chat.completions.create(
    messages = messages,
    model = deployment_name,
    temperature=0,
    top_p = 0.2
)

In [ ]:
print(response.choices[0].message.content)

Question: What is the first step in solving a two-step equation?

Options:
a) Add or subtract the same number from both sides
b) Multiply or divide both sides by the same number
c) Combine like terms on both sides
d) None of the above

Correct Option: b) Multiply or divide both sides by the same number

Explanation:
The first step in solving a two-step equation is to undo the addition or subtraction by multiplying or dividing both sides by the same number.

Incorrect Answers:
a) This is the first step in solving a one-step equation, not a two-step equation.
c) This step comes after the first step in solving a two-step equation.
d) There is a first step in solving a two-step equation.

Question: Solve for x: 3x + 5 = 14

Options:
a) x = 3
b) x = 4
c) x = 9
d) x = 11

Correct Option: b) x = 4

Explanation:
To solve for x, we first subtract 5 from both sides to get 3x = 9. Then, we divide both sides by 3 to get x = 3.

Incorrect Answers:
a) This is not the correct solution to the equation

We can clearly see that most of the answers provided are incorrect. But you can see that it generates the right answer during explanation. Let's try COT and see if we can get better responses.

An LLM can be made to "think" in steps by making small changes to the prompt. The general way of achieving that is by adding the line "think in steps" to the prompt.
Let me show you a small change in our curren prompt which has the same effect. It forces the llm to think first and answer later.

Placing the explanation before the correct option in the prompt is intended to prompt the LLM Model to engage in a more thoughtful process when formulating its response. Rather than immediately providing an answer, this approach encourages the model to approach the problem by breaking it down into simpler sub-problems. By doing so, it is more probable that the model will find answers to these sub-problems within its training data, thereby increasing the likelihood of accurate responses. Subsequently, the model can build upon the solutions obtained from these sub-problems to address the entire problem effectively.

In [ ]:
quiz_input = """Task: Generate a mcq quiz on the teaching material mentioned above.


Output Format:
Use the following format for each question:

Question: [QUESTION TEXT]
Options:
a) [OPTION A]
b) [OPTION B]
c) [OPTION C]
d) [OPTION D]

Explanation:
[EXPLANATION FOR CORRECT ANSWER]

Correct Option: [Correct Option]

Incorrect Answers:
a) [EXPLANATION FOR OPTION A BEING CORRECT/INCORRECT]
b) [EXPLANATION FOR OPTION B BEING CORRECT/INCORRECT]
c) [EXPLANATION FOR OPTION C BEING CORRECT/INCORRECT]
d) [EXPLANATION FOR OPTION D BEING CORRECT/INCORRECT]

"""

In [ ]:
user_input = """
**Lesson Material:**

solving two-step equations"""

In [ ]:
messages = [{"role":"user","content":user_input+quiz_input}]

In [ ]:
response = client.chat.completions.create(
    messages = messages,
    model = deployment_name,
    temperature=0,
    top_p = 0.2
)

In [ ]:
print(response.choices[0].message.content)

Question: What is the first step in solving a two-step equation?
Options:
a) Add or subtract the constant term
b) Multiply or divide the variable term
c) Combine like terms
d) None of the above

Explanation: The first step in solving a two-step equation is to undo the addition or subtraction by performing the opposite operation.

Correct Option: a

Incorrect Answers:
b) Multiplying or dividing the variable term is the second step in solving a two-step equation.
c) Combining like terms is not necessary in solving a two-step equation.
d) None of the above is incorrect as there is a specific first step in solving a two-step equation.

Question: Solve for x: 3x + 5 = 14
Options:
a) x = 3
b) x = 4
c) x = 9
d) x = 11

Explanation: The first step is to undo the addition by subtracting 5 from both sides. This gives 3x = 9. The second step is to undo the multiplication by dividing both sides by 3. This gives x = 3.

Correct Option: a

Incorrect Answers:
b) This is not the correct solution.
c) T



---


The answers are much more accurate.


---



### 4.2 COT

Let's try a more standard CoT approach and let's ask the model to 'think through the problem and explain its answer' without doing the trick with explanation and let's see how good that approach is.

In [ ]:
quiz_input = """Think through and explain your answer
Task: Generate a mcq quiz with 10 questions on the teaching material mentioned above.

Output Format:
Use the following format for each question:

Question: [QUESTION TEXT]
Options:
a) [OPTION A]
b) [OPTION B]
c) [OPTION C]
d) [OPTION D]

Correct Option: [Correct Option]

Explanation:
[EXPLANATION FOR CORRECT ANSWER]

Incorrect Answers:
a) [EXPLANATION FOR OPTION A BEING CORRECT/INCORRECT]
b) [EXPLANATION FOR OPTION B BEING CORRECT/INCORRECT]
c) [EXPLANATION FOR OPTION C BEING CORRECT/INCORRECT]
d) [EXPLANATION FOR OPTION D BEING CORRECT/INCORRECT]

"""

In [ ]:
user_input = """
**Lesson Material:**

solving two-step equations"""

In [ ]:
messages = [{"role":"user","content":user_input+quiz_input}]

In [ ]:
response = client.chat.completions.create(
    messages = messages,
    model = deployment_name,
    temperature=0.2
)

In [ ]:
print(response.choices[0].message.content)

Question: What is the first step in solving a two-step equation?
Options:
a) Add or subtract the constant term
b) Multiply or divide the variable term
c) Combine like terms
d) None of the above

Correct Option: b) Multiply or divide the variable term

Explanation:
The first step in solving a two-step equation is to undo the operation that is being performed on the variable term. If the variable term is being multiplied or divided, then we need to undo that operation by dividing or multiplying, respectively.

Incorrect Answers:
a) Adding or subtracting the constant term is the second step in solving a two-step equation.
c) Combining like terms is not necessary in solving a two-step equation.
d) None of the above is incorrect as there is a specific first step in solving a two-step equation.

Question: Solve for x: 3x + 5 = 14
Options:
a) x = 3
b) x = 4
c) x = 9
d) x = 11

Correct Option: b) x = 4

Explanation:
To solve this equation, we need to undo the addition of 5 to the variable term

It is as bad as a zero-prompt before switching the explanation and correct option placement. We can see that the simple trick of placing explanation above correct answer works much better than a standard CoT trick. This illustrates that prompt engineering is an emperical field and experimentation is the only way to find the best prompt.


Upon repeated attempts at this exercise, you'll find that regardless of the quality of the prompt, language models (LLMs) will inevitably produce incorrect responses. This is because LLMs operate probabilistically and aren't designed for precise calculations. While some LLMs tailored for math might perform better, there's no guarantee of always getting the right answer.

The only reliable method for accurate responses from LLMs is to utilize agents that go beyond what we cover in this course. These agents represent the most suitable LLM technology for such tasks.